In [1]:
cd /home/k64t/person-reid


/home/k64t/person-reid


In [2]:
import transformers
import tensorflow as tf
import pandas as pd
import torch
all_data_path = 'CoLA/data/'
train_path = all_data_path + 'train.tsv'
test_path = all_data_path + 'test.tsv'
val_path = all_data_path + 'dev.tsv'

df = pd.read_csv(train_path, delimiter='\t', header=None, names=['sentence_source', 'label', 'label_notes', 'sentence'])

print('Number of training sentences: {:,}\n'.format(df.shape[0]))

df.sample(10)

2024-06-26 10:17:17.297954: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-06-26 10:17:17.298006: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-06-26 10:17:17.298043: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-06-26 10:17:17.312170: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-06-26 10:17:18.194802: W tensorflow/c

Number of training sentences: 8,551



,sentence_source,label,label_notes,sentence
7771,ad03,1,NaN,Agamemnon seems to have left.
2914,l-93,1,NaN,Smith inscribed his name over the door.
6554,g_81,1,NaN,The kennel which Mary made and Fido sleeps in ...
4904,ks08,1,NaN,They all agreed to include those matters which...
3295,l-93,1,NaN,In the mountains there raged a fire.
1058,bc01,1,NaN,Everyone who had been worrying himself stiff s...
5371,b_73,0,*,Sally eats more the stuff.
7691,sks13,0,*,I sent Bill money to Mary to Sam.
4873,ks08,1,NaN,the student who everyone likes left.
7216,sks13,1,NaN,What John told us is that he wants to quit sch...


In [3]:
sentences = df.sentence.values
labels = df.label.values
from transformers import BertTokenizer

# Load the BERT tokenizer.
print('Loading BERT tokenizer...')
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)

Loading BERT tokenizer...


In [4]:
input_ids = []

for sent in sentences:
    encoded_sent = tokenizer.encode(sent,add_special_tokens = True,)
    input_ids.append(encoded_sent)

print('Original: ', sentences[0])
print('Token IDs:', input_ids[0])

Original:  Our friends won't buy this analysis, let alone the next one we propose.
Token IDs: [101, 2256, 2814, 2180, 1005, 1056, 4965, 2023, 4106, 1010, 2292, 2894, 1996, 2279, 2028, 2057, 16599, 1012, 102]


In [5]:
print('Max sentence length: ', max([len(sen) for sen in input_ids]))

Max sentence length:  47


In [6]:
from keras.preprocessing.sequence import pad_sequences
MAX_LEN = 64

print('\nPadding/truncating all sentences to %d values...' % MAX_LEN)

print('\nPadding token: "{:}", ID: {:}'.format(tokenizer.pad_token, tokenizer.pad_token_id))

input_ids = pad_sequences(input_ids, maxlen=MAX_LEN, dtype="long", 
                          value=0, truncating="post", padding="post")


Padding/truncating all sentences to 64 values...

Padding token: "[PAD]", ID: 0


In [7]:
attention_masks = []

for sent in input_ids:
    att_mask = [int(token_id > 0) for token_id in sent]
    attention_masks.append(att_mask)

In [8]:
from sklearn.model_selection import train_test_split

train_inputs, validation_inputs, train_labels, validation_labels = train_test_split(input_ids, labels, 
                                                            random_state=69, test_size=0.1)
train_masks, validation_masks, _, _ = train_test_split(attention_masks, labels,
                                             random_state=69, test_size=0.1)

In [9]:
train_inputs = torch.tensor(train_inputs)
validation_inputs = torch.tensor(validation_inputs)

train_labels = torch.tensor(train_labels)
validation_labels = torch.tensor(validation_labels)

train_masks = torch.tensor(train_masks)
validation_masks = torch.tensor(validation_masks)

In [10]:
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler

batch_size = 64

train_data = TensorDataset(train_inputs, train_masks, train_labels)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)

validation_data = TensorDataset(validation_inputs, validation_masks, validation_labels)
validation_sampler = SequentialSampler(validation_data)
validation_dataloader = DataLoader(validation_data, sampler=validation_sampler, batch_size=batch_size)

In [11]:
from transformers import BertForSequenceClassification, AdamW, BertConfig

model = BertForSequenceClassification.from_pretrained(
    "bert-base-uncased",
    num_labels = 2, 
    output_attentions = False,
    output_hidden_states = False,
)

model.cuda()

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly i

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12,

In [12]:
optimizer = AdamW(model.parameters(),lr = 2e-5,eps = 1e-8)
from transformers import get_linear_schedule_with_warmup
epochs = 20
total_steps = len(train_dataloader) * epochs
scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps = 0,
                                            num_training_steps = total_steps)
scheduler

/home/k64t/_conda/anaconda3/lib/python3.9/site-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [13]:
import numpy as np
from sklearn.metrics import f1_score
import random

def flat_accuracy_f1(preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()

    f1_flat = f1_score(labels_flat, pred_flat)

    return np.sum(pred_flat == labels_flat) / len(labels_flat), f1_flat

In [14]:
import time
import datetime

def format_time(elapsed):
    elapsed_rounded = int(round((elapsed)))
    return str(datetime.timedelta(seconds=elapsed_rounded))

In [16]:

def eval_dataset(validation_dataloader):
    t0 = time.time()
    model.eval()

    # Tracking variables 
    eval_loss, eval_accuracy = 0, 0
    nb_eval_steps, nb_eval_examples = 0, 0
    eval_f1 = 0

    for batch in validation_dataloader:
        
        # Add batch to GPU
        batch = tuple(t.to(device) for t in batch)
        
        # Unpack the inputs from our dataloader
        b_input_ids, b_input_mask, b_labels = batch
        
        with torch.no_grad():        

            outputs = model(b_input_ids, 
                            token_type_ids=None, 
                            attention_mask=b_input_mask)
        

        logits = outputs[0]

        # Move logits and labels to CPU
        logits = logits.detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()
        
        # Calculate the accuracy for this batch of test sentences.
        tmp_eval_accuracy, tmp_f1 = flat_accuracy_f1(logits, label_ids)
        
        # Accumulate the total accuracy.
        eval_accuracy += tmp_eval_accuracy
        eval_f1 += tmp_f1

        # Track the number of batches
        nb_eval_steps += 1

    # Report the final accuracy for this validation run.
    print("  Accuracy: {0:.2f}".format(eval_accuracy/nb_eval_steps))
    print("  F1_score: {0:.2f}".format(eval_f1/nb_eval_steps))
    print("  Validation took: {:}".format(format_time(time.time() - t0)))


device = 'cuda'
seed_val = 42

random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

loss_values = []

for epoch_i in range(0, epochs):

    print("")
    print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
    print('Training...')

    t0 = time.time()
    total_loss = 0
    model.train()

    for step, batch in enumerate(train_dataloader):

        if step % 40 == 0 and not step == 0:
            elapsed = format_time(time.time() - t0)
            print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(train_dataloader), elapsed))

        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)

        model.zero_grad()        
        outputs = model(b_input_ids, 
                    token_type_ids=None, 
                    attention_mask=b_input_mask, 
                    labels=b_labels)
        
        loss = outputs[0]
        total_loss += loss.item()
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        optimizer.step()
        scheduler.step()

    avg_train_loss = total_loss / len(train_dataloader)            
    loss_values.append(avg_train_loss)

    print("")
    print("  Average training loss: {0:.2f}".format(avg_train_loss))
    print("  Training epcoh took: {:}".format(format_time(time.time() - t0)))

    print("")
    print("Running Validation...")
    eval_dataset(validation_dataloader)
    
print("")
print("Training complete!")



======== Epoch 1 / 20 ========
Training...
  Batch    40  of    121.    Elapsed: 0:00:23.
  Batch    80  of    121.    Elapsed: 0:00:46.
  Batch   120  of    121.    Elapsed: 0:01:10.

  Average training loss: 0.29
  Training epcoh took: 0:01:10

Running Validation...
  Accuracy: 0.83
  F1_score: 0.88
  Validation took: 0:00:03

======== Epoch 2 / 20 ========
Training...
  Batch    40  of    121.    Elapsed: 0:00:24.
  Batch    80  of    121.    Elapsed: 0:00:48.
  Batch   120  of    121.    Elapsed: 0:01:11.

  Average training loss: 0.25
  Training epcoh took: 0:01:12

Running Validation...
  Accuracy: 0.83
  F1_score: 0.88
  Validation took: 0:00:03

======== Epoch 3 / 20 ========
Training...
  Batch    40  of    121.    Elapsed: 0:00:24.
  Batch    80  of    121.    Elapsed: 0:00:48.
  Batch   120  of    121.    Elapsed: 0:01:12.

  Average training loss: 0.17
  Training epcoh took: 0:01:12

Running Validation...
  Accuracy: 0.82
  F1_score: 0.88
  Validation took: 0:00:03

======

In [17]:
# Load the dataset into a pandas dataframe.
df = pd.read_csv(val_path, delimiter='\t', header=None, names=['sentence_source', 'label', 'label_notes', 'sentence'])

print('Number of test sentences: {:,}\n'.format(df.shape[0]))

sentences = df.sentence.values
labels = df.label.values

input_ids = []

for sent in sentences:
    encoded_sent = tokenizer.encode(
                        sent,                      # Sentence to encode.
                        add_special_tokens = True, # Add '[CLS]' and '[SEP]'
                   )
    
    input_ids.append(encoded_sent)

input_ids = pad_sequences(input_ids, maxlen=MAX_LEN, 
                          dtype="long", truncating="post", padding="post")

attention_masks = []

for seq in input_ids:
    seq_mask = [float(i>0) for i in seq]
    attention_masks.append(seq_mask) 

prediction_inputs = torch.tensor(input_ids)
prediction_masks = torch.tensor(attention_masks)
prediction_labels = torch.tensor(labels)

batch_size = 64  

prediction_data = TensorDataset(prediction_inputs, prediction_masks, prediction_labels)
prediction_sampler = SequentialSampler(prediction_data)
prediction_dataloader = DataLoader(prediction_data, sampler=prediction_sampler, batch_size=batch_size)


print('Predicting labels for {:,} test sentences...'.format(len(prediction_inputs)))

model.eval()

eval_accuracy, eval_f1, nb_eval_steps = 0, 0, 0

for batch in prediction_dataloader:
    batch = tuple(t.to(device) for t in batch)
    
    b_input_ids, b_input_mask, b_labels = batch
    
    with torch.no_grad():
        outputs = model(b_input_ids, token_type_ids=None, 
                        attention_mask=b_input_mask)

    logits = outputs[0]
    logits = logits.detach().cpu().numpy()
    label_ids = b_labels.to('cpu').numpy()
    
    tmp_eval_accuracy, tmp_f1 = flat_accuracy_f1(logits, label_ids)
        
    eval_accuracy += tmp_eval_accuracy
    eval_f1 += tmp_f1

    nb_eval_steps += 1

print("  Accuracy: {0:.2f}".format(eval_accuracy/nb_eval_steps))
print("  F1_score: {0:.2f}".format(eval_f1/nb_eval_steps))
print("  Validation took: {:}".format(format_time(time.time() - t0)))

Number of test sentences: 1,043

Predicting labels for 1,043 test sentences...
  Accuracy: 0.82
  F1_score: 0.88
  Validation took: 0:01:18


In [19]:
from pathlib import Path
def save_model(name):
    MODEL_PATH = Path('CoLA/models/')
    MODEL_NAME = Path(name + '.pth')
    MODEL_SAVE_PATH = MODEL_PATH / MODEL_NAME
    #print(f'Saving model to : {MODEL_SAVE_PATH}')
    torch.save(obj = model.state_dict(),
            f = MODEL_SAVE_PATH)

save_model('finetuneBERT_CoLA')